In [ ]:
!pip install numpy torch

In [ ]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# the cost of nutreo-cubes in post-singularity 2024 given the number of neo-automata in the region(per capita)
# the price of paperclips(in keratin clippings)
#quantity of trash in region(trash particles per person)

core_dataset = torch.round(torch.abs(torch.randn(50, 3) * 1000))

print(core_dataset)

In [ ]:
core_output_dataset = torch.round(torch.abs(torch.randn(50,1)) * 1000) 
print(core_output_dataset)

In [ ]:
# define weights
weights = torch.randn(3, 1, requires_grad=True)
print(weights)

In [ ]:
# define bias 
bias = torch.randn(1, requires_grad=True)
print(bias)

In [ ]:
parameters = [weights, bias]


In [ ]:
def model(input_data, parameters):
    return torch.matmul(input_data, parameters[0]) + bias

In [ ]:
def loss_function(predicted_values, actual_values):
    diff = predicted_values - actual_values
    return torch.sum(diff * diff) / diff.numel()

In [ ]:
single_prediction = model(core_dataset, parameters)
print(single_prediction)

In [ ]:
loss = loss_function(single_prediction, core_output_dataset)
print(loss)

In [ ]:
def gradient_descent(parameters, learning_rate):
    for parameter in parameters:
        parameter -= (learning_rate * parameter.grad)

In [ ]:
def reset_parameter_gradient(parameters):
    for parameter in parameters:
        if parameter.grad is not None:
            parameter.grad.zero_()

In [ ]:
# let's take the gradient
loss_2 = loss_function(single_prediction, core_output_dataset)
reset_parameter_gradient(parameters)
loss_2.backward()



In [ ]:
reset_parameter_gradient(parameters)
with torch.no_grad():
    gradient_descent(parameters, 1e-7)
    
reset_parameter_gradient(parameters)



In [ ]:
def train(epoch, parameters, learning_rate, input_data, actual_results):
    for i in range(epoch):
        prediction = model(input_data, parameters)
        loss = loss_function(prediction, actual_results)
        loss.backward()
        with torch.no_grad():
            gradient_descent(parameters, learning_rate)
        reset_parameter_gradient(parameters)
        print(i)
        print(loss)

In [ ]:
train(10000, parameters, 1e-7, core_dataset, core_output_dataset)

In [ ]:
final_loss = loss_function(model(core_dataset,parameters), core_output_dataset)
print(final_loss)

In [ ]:
print(model(core_dataset, parameters))
print(core_output_dataset)

In [ ]:
neural_net = nn.LazyLinear(1)

In [ ]:
print(neural_net.weight)
print(neural_net.bias)

In [ ]:
new_loss_function = nn.MSELoss()

In [ ]:
optimizer = torch.optim.SGD(neural_net.parameters(), 1e-7)

In [ ]:
data_set = TensorDataset(core_dataset, core_output_dataset)

data_loader = DataLoader(data_set, 10, shuffle=True)

In [ ]:
def train_with_imports(epoch, model, learning_rate, dataloader, loss_function):
    optimizer = torch.optim.SGD(model.parameters(), learning_rate, weight_decay=2)
    for i in range(epoch):
        for in_data, out_data in data_loader:
            prediction = model(in_data)
            loss = loss_function(prediction, out_data)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print(i)
        print(loss)
    

In [ ]:
train_with_imports(1000, neural_net, 1e-11, data_loader, new_loss_function)

In [ ]:
def l2_norm(input_data):
    return ((input_data ** 2).sum()) /2


In [ ]:
def loss_function_decay(predicted_values, actual_values, reg_constant, parameters):
    return loss_function(predicted_values, actual_values) + (reg_constant * l2_norm(weights))

In [ ]:

def train_with_weight_decay(epoch, parameters, learning_rate, input_data, actual_results, reg_constant):
    for i in range(epoch):
        prediction = model(input_data, parameters)
        loss = loss_function_decay(prediction, actual_results, reg_constant, parameters)
        loss.backward()
        with torch.no_grad():
            gradient_descent(parameters, learning_rate)
        reset_parameter_gradient(parameters)
        print(i)
        print(loss)

In [ ]:
train_with_weight_decay(100000, parameters, 1e-7, core_dataset, core_output_dataset, 1)